# Прореживание сети 

А теперь проерживание сетки! Как это ни странно, код будет очень похож, за исключением одного нюанса. Нужно удалить, а не добавить столбец/строчку. 

In [ ]:
from IPython import display
from matplotlib import pyplot as plt
%matplotlib inline

# Для визуализации обучения 
def visualize(l1,l2, h1, h2):
    plt.figure(figsize=(20,5)) 
    epo_range = range(1,len(h1)+1)
    tick_range = range(1,len(h1)+1,2)
    plt.subplot(1,2,1)
    plt.title('Learning loss')
    plt.plot(epo_range,l1, label='train set')
    plt.plot(epo_range,l2, label='valid set')
    plt.grid()
    plt.xticks(tick_range)
    plt.legend(title = 'Loss at:')
    
    plt.subplot(1,2,2)
    plt.title('Learning accuracy')
    plt.plot(epo_range,h1, label='train set')
    plt.plot(epo_range,h2, label='valid set')
    plt.grid()
    plt.xticks(tick_range)
    plt.ylim(0, 1.)
    plt.legend(title = 'Accuracy at:')
    
    display.clear_output(wait=True)
    plt.show()

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras  

import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split

# внутри keras уже есть набор данных, подгрузим его 
def load_dataset( ):
    (X_train, y_train), (X_test, y_test) = keras.datasets.mnist.load_data()

    # нормализация матриц
    X_train = X_train.astype(np.float32) / 255.
    X_test = X_test.astype(np.float32) / 255.
    
    # вытягиваем матрицы [n,28,28] в вектрора [n,28**2]
    X_train = X_train.reshape(X_train.shape[0],28**2)
    X_test = X_test.reshape(X_test.shape[0],28**2)
    
    # сделаем OHE для таргета
    y_train = keras.utils.to_categorical(y_train).astype(np.float32)
    y_test = keras.utils.to_categorical(y_test).astype(np.float32)

    # последние 10000 примеров из трэйна оставим для валидации
    X_train, X_val, y_train, y_val =  train_test_split(X_train, y_train, test_size=0.15, stratify=y_train)
    
    return X_train, y_train, X_val, y_val, X_test, y_test

X_train, y_train, X_val, y_val, X_test, y_test = load_dataset()

Свой генератор батчей. Как думаете, что с ним не так? Исправьте!

In [ ]:
def batches_generator(X, y, batch_size):
    n_batches = int(X.shape[0]/batch_size) + 1
    for batch_idx in range(n_batches):
        indices = (batch_idx*batch_size, min(X.shape[0], (batch_idx+1)*batch_size))
        yield X[indices[0]:indices[1]], y[indices[0]:indices[1]]

Собираем своего монстра. 

In [ ]:
class model_simple():
    
    def __init__(self, L1, L2, L3, L4, L5, learning_rate):
        
        self.W1 = tf.Variable(tf.random.truncated_normal([784, L1], stddev=0.1))
        self.b1 = tf.Variable(tf.zeros([L1]))

        self.W2 = tf.Variable(tf.random.truncated_normal([L1, L2], stddev=0.1))
        self.b2 = tf.Variable(tf.zeros([L2]))

        self.W3 = tf.Variable(tf.random.truncated_normal([L2, L3], stddev=0.1))
        self.b3 = tf.Variable(tf.zeros([L3]))
        
        self.W4 = tf.Variable(tf.random.truncated_normal([L3, L4], stddev=0.1))
        self.b4 = tf.Variable(tf.zeros([L4]))
        
        self.W5 = tf.Variable(tf.random.truncated_normal([L4, L5], stddev=0.1))
        self.b5 = tf.Variable(tf.zeros([L5]))
        
        self.opt = tf.optimizers.Adam(learning_rate)
              
    def generate_prediction_logits(self,X_train):
        Y1 = tf.nn.relu(tf.matmul(X_train, self.W1) + self.b1)
        Y2 = tf.nn.relu(tf.matmul(Y1, self.W2) + self.b2)
        Y3 = tf.nn.relu(tf.matmul(Y2, self.W3) + self.b3)
        Y4 = tf.nn.relu(tf.matmul(Y3, self.W4) + self.b4)
        Ylogits = tf.matmul(Y4, self.W5) + self.b5
        return Ylogits
    
    def make_loss(self,X_train,y_train):
        Ylogits = self.generate_prediction_logits(X_train)
        cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=Ylogits, labels=y_train)
        return tf.reduce_mean(cross_entropy)
    
    def make_learning_step(self,X_train,y_train):
        # находим loss и пробрасываем градиент
        with tf.GradientTape() as g:
            loss = self.make_loss(X_train, y_train)
        gradients = g.gradient(loss, [self.W1,self.W2,self.W3,self.b1, self.b2,
                                     self.b3])
        self.opt.apply_gradients(zip(gradients, [self.W1,self.W2,self.W3,self.b1, self.b2,
                                     self.b3]))
        return gradients
    
    def model_acc(self,x_train,y_train):
        Ylogits = self.generate_prediction_logits(x_train)
        Y = tf.nn.softmax(Ylogits)
        correct_prediction = tf.equal(tf.argmax(Y, 1), tf.argmax(y_train, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        return accuracy

__Вопрос:__ куда надо добавить декоратор, чтобы училось более бысто?

In [ ]:
L1 = 100
L2 = 200
L3 = 300
L4 = 350
L5 = 10
 
learning_rate = 0.003
Our_Model = model_simple(L1,L2,L3,L4,L5,learning_rate)

In [ ]:
n_epochs = 30
batch_size = 100
loss_test, loss_train  = [ ], [ ] 
acc_test, acc_train = [ ], [ ]


for i in tqdm(range(n_epochs)):
    
    # число батчей
    num_batches = (X_train.shape[0] / batch_size) + 1
    batch_gen = batches_generator(X_train, y_train, batch_size)
    
    # пошёл цикл по батчам 
    for X_batch, y_batch in batch_gen:
        Our_Model.make_learning_step(X_batch,y_batch)

    # ищем метрики в конце эпохи
    loss_train.append(Our_Model.make_loss(X_train,y_train))
    loss_test.append(Our_Model.make_loss(X_val,y_val))
    
    acc_train.append(Our_Model.model_acc(X_train,y_train))
    acc_test.append(Our_Model.model_acc(X_val,y_val))

    visualize(loss_train, loss_test, acc_train, acc_test) 

Посмотреть что за веса получилсь можно вот так: 

In [ ]:
Our_Model.W5

In [ ]:
Our_Model.W5.shape

Давайте сохраним получившиеся веса, например в виде numpy массивов, чтобы в случае чего мы могли бы их подгрузить. Модели можно сохранять ещё и как графы вычислений. Но про это мы поговорим позже. 

Ежу понятно, что копипста строчек для сохранения матриц - не самая хорошая идея. Чтобы не заниматься такой ерундой, нужно унаследовать класс, в котором мы задаём модель, от Model. Но об этом мы поговорим в следующий раз. Сейчас пока что мы акцентируем внимание именно на том, что нейросетка - это просто поток из кучи матриц. 

In [ ]:
weight_1_save = Our_Model.W1.numpy()
bias_1_save = Our_Model.b1.numpy()
np.save('weight_1_save.npy', weight_1_save)
np.save('bias_1_save.npy', bias_1_save)

weight_2_save = Our_Model.W2.numpy()
bias_2_save = Our_Model.W2.numpy()
np.save('weight_2_save.npy', weight_2_save)
np.save('bias_2_save.npy', bias_2_save)

weight_3_save = Our_Model.W3.numpy()
bias_3_save = Our_Model.W3.numpy()
np.save('weight_3_save.npy', weight_3_save)
np.save('bias_3_save.npy', bias_3_save)

weight_4_save = Our_Model.W4.numpy()
bias_4_save = Our_Model.W4.numpy()
np.save('weight_4_save.npy', weight_4_save)
np.save('bias_4_save.npy', bias_4_save)

weight_5_save = Our_Model.W5.numpy()
bias_5_save = Our_Model.W5.numpy()
np.save('weight_5_save.npy', weight_5_save)
np.save('bias_5_save.npy', bias_5_save)

In [ ]:
# загрузить веса можно следующим способом
bias_1_save = np.load('bias_1_save.npy')

Что значит на матричном уровне, что нейрон отключен? Как можно проверить, что стало лучше или хуже? 

In [ ]:
weight_1 = Our_Model.W1.numpy()
bias_1 = Our_Model.b1.numpy() # загрузили веса

# посчитаем наш базовый лосс на валидации
loss_base = ...  # ВАШ КОД! 

Проведем следующую процедуру-отключаем (зануляем) каждый нейрон, смотрим насколько у нас меняется итоговый loss на валидации. Это изменение - "значимость" каждого нейрона. Давайте по этой характеристике отранжируем нейроны. Самый незначимый будем выбрасывать из сетки.

In [ ]:
change_in_loss = [] # сохраним лист с изменениями

for i in tqdm(range(weight_1.shape[1])):
    
    weight_1=weight_1_save.copy() 
    bias_1=bias_1_save.copy() # зачем мы копируем ? 
    
    weight_1[:,i] = 0
    bias_1[i] = 0
    Our_Model.W1 = weight_1
    Our_Model.b1 = bias_1
    
    # наш новый loss с изменениями
    loss_new = Our_Model.make_loss(X_val,y_val).numpy() 
    change_in_loss.append(loss_new-loss_base)

In [ ]:
import pandas as pd
pd.DataFrame({'neural_list':list(range(weight_1.shape[1])),
             'change_in_loss':change_in_loss}).sort_values('change_in_loss')

In [ ]:
# теперь перепишите нейронную сеть, как она должна выглядеть
# с исключенным нейронном и поучите еще пару десятков эпох
# Удалить строки из numpy можно таким образом
# иницализируем нейроны прошлыми результатами
# если они где-то потерялись, можно их загрузить
# weight_1=np.delete(weight_1_save,axis=0,obj=24) -удалит 24 строку
# weight_1=np.delete(weight_1_save,axis=1,obj=24) -удалит 24 столбец
# подсказка-следите за размерностями!

In [ ]:
# Ваш код

## Домашка

Сейчас мы сделали добавление нового нейрона на слой довольно убого. Мы просто-напросто пересоздали класс и добавили в него новые веса. Это такое себе занятие. Хочется, чтобы можно было добавлять новый нейрон на конкретный слой с помощью какого-нибудь специального метода, который был бы задан прямо внутри класса. По аналогии хотелось бы увидеть метод, который удаляет нейрон. 

* Реализуйте внутри нашего класса метод для добавления/удаления нейронов 
* Подумайте о том как именно можно было бы выбирать нейрон для удаления из сетки (как можно попытаться вычислить значимость каждого конкретного нейрона и выбрать для удаления наименее значимый).
* Подумайте как реализовать подсчёт значимости в виде метода и добавьте его в модель. 

__Бонусный трэк для самых упоротых:__ 

* Реализуйте метод, который добавляет в нейронку новый слой
* Попробуйте подобрать для MNIST оптимальную архитектуру жадным алгоритмом с пошаговым добавлением/выкидыванием нейронов и слоёв. Самостоятельно придумайте правила перебора и способ останавливать его. 
